# This file is used to break CSVs into individual chunks. 

In [1]:
import pandas as pd
import json
import os

In [ ]:
os.getcwd()

In [1]:
# Define directories, specific paths truncated for security reasons. 
csv_files = [csv_files]
output_dirs = [output_d
metadata_dirs = [dirs_to_files]

# Process each CSV and create corresponding output and metadata
for csv_file, output_dir, metadata_dir in zip(csv_files, output_dirs, metadata_dirs):
    df = pd.read_csv(csv_file)
    
    # Ensure the output and metadata directories exist
    os.makedirs(output_dir, exist_ok=True)
    os.makedirs(metadata_dir, exist_ok=True)
    
    # Iterate through each row in the dataframe
    for index, row in df.iterrows():
        # Extract ign and ensure it's usable as a filename
        player_ign = str(row.get("ign", f"player_{index + 1}")).replace(" ", "_").lower()
        
        # Extract player data into a new dataframe
        player_df = pd.DataFrame([row])
        
        # Save as a new CSV file, naming it after the player's ign
        csv_filename = f"{player_ign}.csv"
        player_df.to_csv(os.path.join(output_dir, csv_filename), index=False)
        
        # Create corresponding metadata JSON
        metadata = {
            "metadataAttributes": {
                "standings": str(row.get("standings", "")),
                "igl": str(row.get("igl", "")).lower(),  # Convert to lowercase for consistency
                "region": "amer",  # Set region to "amer"
                "league": str(row.get("league", "vct game changers"))  # Added league for metadata
            }
        }
        
        # Save metadata as a JSON file, also named after the player's ign
        metadata_filename = f"{player_ign}.csv.metadata.json"
        with open(os.path.join(metadata_dir, metadata_filename), 'w') as json_file:
            json.dump(metadata, json_file, indent=4)

print("CSV and metadata files created for all players")

CSV and metadata files created for all players, named after IGN, with region set to 'amer'.
